# Basic implementation of Rag

In [1]:
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from dotenv import load_dotenv

load_dotenv()

True

### Function to Retrieve a bible text from api and combine it as one text chunk

In [2]:
import requests
import re

## get bible passage from https://bible-api and return a string version of the bible

def bible_text(passage: str) -> str:
    """
    Fetches a Bible passage from Bible-API and returns it in a formatted string:
    Example: "Luke 15:1-2 1. Text 2. Text"
    """
    url = f"https://bible-api.com/{passage.replace(' ', '%20')}?translation=kjv"
    response = requests.get(url)
    
    if response.status_code != 200:
        return f"Error fetching passage: {response.status_code}"

    data = response.json()
    reference = data.get("reference", "")
    verses = data.get("verses", [])
    
    merged_verses = " ".join(
        [f'{v["verse"]}. {v["text"].strip()}' for v in verses]
    )
    final_text = f"{reference} {merged_verses}"
    cleaned_text =re.sub(r'\s+', ' ', final_text).strip()

    return cleaned_text



### Retrieve the text using the bible_text function and store it as a list of Documents

In [3]:
docs =[
Document(page_content=bible_text("Luke 15"), metadata={"source":"Luke 15"}),
Document(page_content=bible_text("Psalm 1"), metadata={"source":"Psalm 1"}),
Document(page_content=bible_text("Psalm 124"), metadata={"source":"Psalm 124"}),
Document(page_content=bible_text("Psalm 2"), metadata={"source":"Psalm 2"}),

] 


### creat an embedding function, create an embedding database db, using the function and the document as parameters 

In [4]:
embedding_function = OpenAIEmbeddings()
db = Chroma.from_documents(docs, embedding_function)

### Create a retriever configuration using the embedded db, then retrieve a question to get a set of similar result
- it is from this 3 results (k=3) that you will now query (invoke) the final answer

In [35]:

retriever =db.as_retriever(search_type="mmr", search_kwargs={"k":3})

question= "what happened when the boy spent all he had"
try:
    result = retriever.invoke(question)
    # print(result)  # uncomment this if you want to see the response- this is a list of document excerpts with similarity to the question answer
except Exception as e:
    print(e)

In [38]:
### generate a prompt template that will be used to finally query the  result (list of similar answers)

In [39]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")  # get the llm


template = """Answer the question based on the following context: {context} 
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)  # create the promptTemplate

# function to convert the generated docs into one big chunk of text
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

#this creates the chain, retreives the context and question using the previous doc response and the previous answer
# this chain will use llm to only retreive result from the previous docs (context) (which is now merged into a big chunk of text with format_docs)
qa_chain = (
    {
        "context": lambda x: format_docs(result),
        "question": lambda x: x
    }
    | prompt
    | llm
    | StrOutputParser()

)

In [42]:
# while True:
#     final_answer=qa_chain.invoke(question)
user_input=question
from IPython.display import display, Markdown
while True:
    if user_input.lower() in ["exit","end","break"]:
        break;
    final_answer = qa_chain.invoke(user_input)
    # print(final_answer)
    print(display(Markdown(final_answer)))
    user_input = input("Question: ")


When the younger son spent all he had, a mighty famine arose in the land, and he began to be in want. He then joined himself to a citizen of that country, who sent him into his fields to feed swine. The son was so hungry that he longed to fill his belly with the husks that the swine ate, but no one gave him anything. This situation led him to realize his dire circumstances and decide to return to his father, acknowledging his mistakes and asking for forgiveness.

None


Question:  List each parable and give each one a title


Based on the context provided from Luke 15, there are three parables presented. Here is a summary of each parable along with a suggested title:

1. **Parable of the Lost Sheep**:  
   Title: "The Lost Sheep"  
   Summary: A shepherd leaves ninety-nine sheep in the wilderness to search for one lost sheep. Upon finding it, he rejoices and calls his friends and neighbors to celebrate. This parable emphasizes the joy in heaven over one sinner who repents.

2. **Parable of the Lost Coin**:  
   Title: "The Lost Coin"  
   Summary: A woman searches diligently for one lost coin out of ten. When she finds it, she calls her friends and neighbors to rejoice with her. This parable illustrates the joy in the presence of the angels over a sinner who repents.

3. **Parable of the Prodigal Son**:  
   Title: "The Prodigal Son"  
   Summary: A younger son demands his inheritance, leaves home, and wastes his wealth in a far country. Facing hardship, he returns home, repents, and is warmly welcomed by his father, who celebrates his return. The parable also describes the reaction of the elder son, who feels neglected. The theme centers on forgiveness, redemption, and the joy of a lost soul returning home.

None


...

None


Question:  why will God laugh at them


The context you provided does not explicitly mention God laughing at anyone. However, the concept of God laughing at the wicked or the ungodly is found in other parts of the Bible, such as Psalms 2:4 and Psalms 37:13. In these verses, the reason for God's laughter is generally because he is sovereign and knows that their plans or plots against the righteous or against His purposes will ultimately fail. God’s laughter is an expression of His supreme power and the futility of opposition against His will.

None


Question:  end


In [41]:
question

'what happened when the boy spent all he had'